In [1]:
import pandas as pd
col_names = ['word', 'pos']
data = pd.read_csv('./Train_utf16.ner', sep=' ', header=None, encoding='utf-16', names=col_names, \
                   skip_blank_lines=False).fillna('')
data.head(30)

,word,pos
0,海,N
1,內,N
2,外,N
3,關,N
4,注,N
5,的,N
6,一,N
7,九,N
8,九,N
9,七,N


In [2]:
data['pos'] = data.apply(lambda row: 'O' if row['pos'] == 'N' else row['pos'], axis=1)
data.head(30)

,word,pos
0,海,O
1,內,O
2,外,O
3,關,O
4,注,O
5,的,O
6,一,O
7,九,O
8,九,O
9,七,O


In [3]:
data.to_csv('./train_total.data', header=None, index=False, sep=' ', encoding='utf-8')

In [15]:
data.iloc[0].name

0

In [47]:
data.tail(5)

,word,pos
1808531,謀,O
1808532,殺,O
1808533,案,O
1808534,。,O
1808535,,


In [48]:
from sklearn.cross_validation import train_test_split
import numpy as np
y = np.zeros(len(data))
# train, test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)
limit = int(len(y) * 0.7)
line = -1
def findSplitPoint(row):
    global line
    if line == -1 and int(row.name) >= limit - 1 and row['word'] == '':
        line = int(row.name)
data.apply(findSplitPoint, axis=1)
print (line)

1266021


In [49]:
train = data[0:line+1]
test = data[line+1:]

In [50]:
print (len(train), len(test), len(data))

1266022 542514 1808536


In [51]:
train.tail(5)

,word,pos
1266017,線,O
1266018,駛,O
1266019,經,O
1266020,。,O
1266021,,


In [52]:
test.head(5)

,word,pos
1266022,前,O
1266023,往,O
1266024,新,B-LOC
1266025,界,I-LOC
1266026,如,O


In [53]:
train.to_csv('./train.data', header=None, index=False, sep=' ', encoding='utf-8')

In [54]:
test.to_csv('./test.data', header=None, index=False, sep=' ', encoding='utf-8')

In [55]:
# pred.data
import pandas as pd
col_names = ['word', 'pos']
data_pred = pd.read_csv('./Test_utf16.ner', sep=' ', header=None, encoding='utf-16', names=col_names, \
                   skip_blank_lines=False).fillna('')

In [56]:
data_pred['pos'] = data_pred.apply(lambda row: '' if row['word'] == '' else 'O', axis=1)
data_pred.head(20)

,word,pos
0,這,O
1,是,O
2,一,O
3,次,O
4,意,O
5,義,O
6,重,O
7,大,O
8,的,O
9,宏,O


In [57]:
data_pred.to_csv('./pred.data', header=None, index=False, sep=' ', encoding='utf-8')

In [ ]:
# train
# crf_learn -f 4  -p 8 -c 3 template1 train.data model > train.rst
# test
# crf_test -m model test.data > test.rst

In [28]:
# eval
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)
test_rst.head(5)

,word,gold,pred
0,彈,N,N
1,的,N,N
2,爆,N,N
3,炸,N,N
4,碎,N,N


In [36]:
# precision, recall, f1
rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.955028911493
Recall: 0.885633769741
F1 0.919023201503


In [37]:
# eval model2 c3
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test2.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.962363647962
Recall: 0.885712049159
F1 0.922448230882


In [40]:
# eval model2_c1
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test2_c1.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.963877673739
Recall: 0.878334213977
F1 0.919119830437


In [42]:
# eval model2_c5
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test2_c5.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.958717916298
Recall: 0.890330534844
F1 0.923259565919


In [43]:
# eval model3_c5
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('./test3.rst', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'N', axis=1)
gold_nums = test_rst['gold'] != 'N' 
pred_nums = test_rst['pred'] != 'N'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print 'Precision:', p
print 'Recall:', r
print 'F1', 2*p*r / (p+r)

Precision: 0.958357006081
Recall: 0.891289457719
F1 0.923607308714


In [52]:
# eval model3_c5 jian
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('/home/derc/workspace/NER/CrfNER/jian/test.rst.jian', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'O', axis=1)
gold_nums = test_rst['gold'] != 'O' 
pred_nums = test_rst['pred'] != 'O'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print ('Precision:', p)
print ('Recall:', r)
print ('F1', 2*p*r / (p+r))

Precision: 0.964161603241
Recall: 0.885096389079
F1 0.92293877551


In [53]:
test_rst.to_csv('/home/derc/workspace/NER/CrfNER/jian/test.rst.jian_con', header=None, index=False, sep=' ', encoding='utf-8')

In [43]:
# eval neural model
col_names = ['word', 'gold', 'pred']
test_rst = pd.read_csv('/home/derc/workspace/NER/ChineseNER/jian/results/ner_predict.utf8', header=None, sep='\s+', encoding='utf-8', names=col_names)

rows = test_rst.apply(lambda row: row['gold'] == row['pred'] and row['pred'] != 'O', axis=1)
gold_nums = test_rst['gold'] != 'O' 
pred_nums = test_rst['pred'] != 'O'
p = 1.0 * rows.sum() / pred_nums.sum()
r = 1.0 * rows.sum() / gold_nums.sum()
print ('Precision:', p)
print ('Recall:', r)
print ('F1', 2*p*r / (p+r))

Precision: 0.0
Recall: nan
F1 nan


/home/derc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [44]:
# compare totoal with 70%data
import pandas as pd
col_names = ['word', 'gold', 'pred']
tot = pd.read_csv('../results_total/ner_predict.utf8', header=None, names=col_names, sep='\s+', encoding='utf-8')

pat = pd.read_csv('../results_pred/ner_predict.utf8', header=None, names=col_names, sep='\s+', encoding='utf-8')

In [61]:
# compare totoal with 70%data
import pandas as pd
col_names = ['word', 'gold', 'pred']
pat = pd.read_csv('../jian/results/ner_predict.utf8', header=None, names=col_names, sep='\s+', encoding='utf-8', skip_blank_lines=False).fillna('')

tot = pd.read_csv('../jian_total/results/ner_predict.utf8', header=None, names=col_names, sep='\s+', encoding='utf-8', skip_blank_lines=False).fillna('')

crf_pat = pd.read_csv('../../CrfNER/jian/pred.rst.jian', header=None, names=col_names, sep='\s+', encoding='utf-8', skip_blank_lines=False).fillna('')

crf = pd.read_csv('../../CrfNER/jian/pred_total.rst.jian', header=None, names=col_names, sep='\s+', encoding='utf-8', skip_blank_lines=False).fillna('')

In [62]:
# 替换原来的汉字
org = pd.read_csv('/home/derc/workspace/NER/CrfNER/data/pred.data.rst', header=None, names=col_names, sep='\s+', encoding='utf-8', skip_blank_lines=False).fillna('')

# cnt = 0
# for idx in range(len(org)):
#     if idx > 0 and org.iloc[idx-1, 0] == '' and org.iloc[idx, 0] == '':
#         cnt += 1
# print (cnt, len(tot) - len(org))

# for idx in range(len(tot)):
#     if tot.iloc[idx,0] != org.iloc[idx,0]:
#         print (idx, tot.iloc[idx,:].values, org.iloc[idx,:].values)
#         break

print (len(tot), len(org))
tot['word'] = org['word']
# tot = tot.insert(0, 'org_word', org['word'])

# # 替换O成N
tot['pred'] = tot.apply(lambda row: 'N' if row['pred'] == 'O' else row['pred'], axis=1)




390540 390540


In [63]:
# 恢复丢掉的空行
import pandas as pd
pred_data = pd.read_csv('/home/derc/workspace/NER/ChineseNER/data/pred.data', header=None, names=col_names, sep='\s+', encoding='utf-8', skip_blank_lines=False).fillna('')

In [56]:
pred_data.head(5)

,word,gold,pred
0,這,O,
1,是,O,
2,一,O,
3,次,O,
4,意,O,


In [64]:
blank_line = pd.DataFrame({'word':'', 'gold':'', 'pred':''}, index=[0])
blank_line

,gold,pred,word
0,,,


In [65]:
cnt = 0
for idx in range(len(pred_data)):
    if idx > 0 and pred_data.iloc[idx,0] == '' and pred_data.iloc[idx-1,0] == '':
        print (pred_data.iloc[idx,:].values)
        cnt += 1
        tot = pd.concat([tot.loc[:idx-1], blank_line, tot.loc[idx:]], ignore_index=True)
print (cnt)


# 删除gold

tot[['word', 'pred']].to_csv('../jian_total/results/ner_predict.utf16', header=None, index=False, sep=' ', encoding='utf-16')

['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
['' '' '']
21


In [51]:
print (len(tot))

390561


In [60]:
tot

,pred,word
0,N,這
1,N,是
2,N,一
3,N,次
4,N,意


In [21]:
org.head(5)

,word,glod
這,O,O
是,O,O
一,O,O
次,O,O
意,O,O


In [63]:
dif = 0
for idx in range(len(tot)):
    if crf.iloc[idx,2] != tot.iloc[idx,2]:
        print (crf.iloc[idx,:].values, tot.iloc[idx,:].values)
        dif += 1
print (dif) 

['丽' 'O' 'B-LOC'] ['丽' 'O' 'B-ORG']
['江' 'O' 'I-LOC'] ['江' 'O' 'I-ORG']
['朝' 'O' 'O'] ['朝' 'O' 'B-LOC']
['阳' 'O' 'O'] ['阳' 'O' 'I-LOC']
['区' 'O' 'O'] ['区' 'O' 'I-LOC']
['画' 'O' 'B-ORG'] ['画' 'O' 'O']
['院' 'O' 'I-ORG'] ['院' 'O' 'O']
['出' 'O' 'B-ORG'] ['出' 'O' 'O']
['版' 'O' 'I-ORG'] ['版' 'O' 'O']
['局' 'O' 'I-ORG'] ['局' 'O' 'O']
['杂' 'O' 'O'] ['杂' 'O' 'B-ORG']
['志' 'O' 'O'] ['志' 'O' 'I-ORG']
['社' 'O' 'O'] ['社' 'O' 'I-ORG']
['会' 'O' 'O'] ['会' 'O' 'B-LOC']
['展' 'O' 'O'] ['展' 'O' 'I-LOC']
['多' 'O' 'O'] ['多' 'O' 'B-LOC']
['渠' 'O' 'O'] ['渠' 'O' 'I-LOC']
['道' 'O' 'O'] ['道' 'O' 'I-LOC']
['劳' 'O' 'O'] ['劳' 'O' 'B-ORG']
['动' 'O' 'O'] ['动' 'O' 'I-ORG']
['报' 'O' 'O'] ['报' 'O' 'I-ORG']
['东' 'O' 'O'] ['东' 'O' 'B-LOC']
['中' 'O' 'O'] ['中' 'O' 'I-LOC']
['西' 'O' 'O'] ['西' 'O' 'I-LOC']
['小' 'O' 'O'] ['小' 'O' 'B-LOC']
['城' 'O' 'O'] ['城' 'O' 'I-LOC']
['镇' 'O' 'O'] ['镇' 'O' 'I-LOC']
['小' 'O' 'O'] ['小' 'O' 'B-LOC']
['城' 'O' 'O'] ['城' 'O' 'I-LOC']
['镇' 'O' 'O'] ['镇' 'O' 'I-LOC']
['小' 'O' 'O'] ['小' 'O' 'B-LOC']


['海' 'O' 'O'] ['海' 'O' 'I-PER']
['美' 'O' 'O'] ['美' 'O' 'I-PER']
['胡' 'O' 'O'] ['胡' 'O' 'B-PER']
['月' 'O' 'O'] ['月' 'O' 'I-PER']
['胡' 'O' 'O'] ['胡' 'O' 'B-PER']
['月' 'O' 'O'] ['月' 'O' 'I-PER']
['爱' 'O' 'B-PER'] ['爱' 'O' 'O']
['谭' 'O' 'I-PER'] ['谭' 'O' 'B-PER']
['美' 'O' 'O'] ['美' 'O' 'B-LOC']
['小' 'O' 'O'] ['小' 'O' 'B-LOC']
['海' 'O' 'O'] ['海' 'O' 'I-LOC']
['美' 'O' 'O'] ['美' 'O' 'I-LOC']
['谭' 'O' 'O'] ['谭' 'O' 'B-PER']
['海' 'O' 'O'] ['海' 'O' 'I-PER']
['美' 'O' 'O'] ['美' 'O' 'I-PER']
['中' 'O' 'O'] ['中' 'O' 'B-ORG']
['石' 'O' 'O'] ['石' 'O' 'I-ORG']
['油' 'O' 'O'] ['油' 'O' 'I-ORG']
['中' 'O' 'O'] ['中' 'O' 'B-ORG']
['石' 'O' 'O'] ['石' 'O' 'I-ORG']
['油' 'O' 'O'] ['油' 'O' 'B-LOC']
['气' 'O' 'O'] ['气' 'O' 'I-LOC']
['田' 'O' 'O'] ['田' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-ORG']
['冶' 'O' 'O'] ['冶' 'O' 'I-ORG']
['晋' 'O' 'O'] ['晋' 'O' 'B-ORG']
['富' 'O' 'O'] ['富' 'O' 'I-ORG']
['汤' 'O' 'O'] ['汤' 'O' 'B-LOC']
['加' 'O' 'O'] ['加' 'O' 'I-LOC']
['汤' 'O' 'O'] ['汤' 'O' 'B-LOC']
['加' 'O' 'O'] ['加' 'O' 'I-LOC']
['红'

['尔' 'O' 'I-PER'] ['尔' 'O' 'I-LOC']
['木' 'O' 'I-PER'] ['木' 'O' 'I-LOC']
['兹' 'O' 'I-PER'] ['兹' 'O' 'I-LOC']
['海' 'O' 'O'] ['海' 'O' 'I-LOC']
['峡' 'O' 'O'] ['峡' 'O' 'I-LOC']
['伯' 'O' 'O'] ['伯' 'O' 'B-PER']
['利' 'O' 'O'] ['利' 'O' 'I-PER']
['恒' 'O' 'O'] ['恒' 'O' 'I-PER']
['马' 'O' 'O'] ['马' 'O' 'I-PER']
['拉' 'O' 'O'] ['拉' 'O' 'I-PER']
['松' 'O' 'O'] ['松' 'O' 'I-PER']
['伯' 'O' 'O'] ['伯' 'O' 'B-PER']
['利' 'O' 'O'] ['利' 'O' 'I-PER']
['恒' 'O' 'O'] ['恒' 'O' 'I-PER']
['越' 'O' 'O'] ['越' 'O' 'B-LOC']
['拉' 'O' 'O'] ['拉' 'O' 'I-LOC']
['越' 'O' 'O'] ['越' 'O' 'I-LOC']
['美' 'O' 'O'] ['美' 'O' 'B-PER']
['满' 'O' 'O'] ['满' 'O' 'I-PER']
['幸' 'O' 'O'] ['幸' 'O' 'I-PER']
['福' 'O' 'O'] ['福' 'O' 'I-PER']
['新' 'O' 'O'] ['新' 'O' 'B-LOC']
['绿' 'O' 'O'] ['绿' 'O' 'I-LOC']
['婀' 'O' 'O'] ['婀' 'O' 'B-PER']
['娜' 'O' 'O'] ['娜' 'O' 'I-PER']
['切' 'O' 'B-PER'] ['切' 'O' 'B-LOC']
['尔' 'O' 'I-PER'] ['尔' 'O' 'I-LOC']
['诺' 'O' 'I-PER'] ['诺' 'O' 'I-LOC']
['贝' 'O' 'I-PER'] ['贝' 'O' 'I-LOC']
['利' 'O' 'I-PER'] ['利' 'O' 'I-LOC']
['劳' 'O'

['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-ORG']
['阿' 'O' 'O'] ['阿' 'O' 'I-ORG']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['举' 'O' 'I-LOC'] ['举' 'O' 'O']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-LOC']
['阿' 'O' 'O'] ['阿' 'O' 'I-LOC']
['中' 'O'

['大' 'O' 'O'] ['大' 'O' 'I-ORG']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['省' 'O' 'O'] ['省' 'O' 'I-LOC']
['建' 'O' 'O'] ['建' 'O' 'B-ORG']
['设' 'O' 'O'] ['设' 'O' 'I-ORG']
['厅' 'O' 'O'] ['厅' 'O' 'I-ORG']
['赖' 'O' 'O'] ['赖' 'O' 'B-PER']
['爱' 'O' 'O'] ['爱' 'O' 'I-PER']
['光' 'O' 'O'] ['光' 'O' 'I-PER']
['郑' 'O' 'O'] ['郑' 'O' 'B-PER']
['功' 'O' 'O'] ['功' 'O' 'I-PER']
['成' 'O' 'O'] ['成' 'O' 'I-PER']
['诚' 'O' 'O'] ['诚' 'O' 'B-ORG']
['实' 'O' 'O'] ['实' 'O' 'I-ORG']
['台' 'O'

...

['西' 'O' 'O'] ['西' 'O' 'I-ORG']
['队' 'O' 'O'] ['队' 'O' 'I-ORG']
['卡' 'O' 'O'] ['卡' 'O' 'B-PER']
['卡' 'O' 'O'] ['卡' 'O' 'I-PER']
['波' 'O' 'B-LOC'] ['波' 'O' 'B-PER']
['恩' 'O' 'I-LOC'] ['恩' 'O' 'I-PER']
['赛' 'O' 'I-PER'] ['赛' 'O' 'O']
['两' 'O' 'O'] ['两' 'O' 'B-ORG']
['大' 'O' 'O'] ['大' 'O' 'I-ORG']
['党' 'O' 'O'] ['党' 'O' 'I-ORG']
['立' 'O' 'O'] ['立' 'O' 'B-ORG']
['院' 'O' 'O'] ['院' 'O' 'I-ORG']
['道' 'O' 'I-PER'] ['道' 'O' 'O']
['德' 'O' 'I-PER'] ['德' 'O' 'B-LOC']
['光' 'O' 'I-PER'] ['光' 'O' 'I-LOC']
['环' 'O' 'I-PER'] ['环' 'O' 'I-LOC']
['林' 'O' 'O'] ['林' 'O' 'B-PER']
['浊' 'O' 'O'] ['浊' 'O' 'I-PER']
['水' 'O' 'O'] ['水' 'O' 'I-PER']
['立' 'O' 'O'] ['立' 'O' 'B-ORG']
['院' 'O' 'O'] ['院' 'O' 'I-ORG']
['佘' 'O' 'O'] ['佘' 'O' 'B-PER']
['契' 'O' 'O'] ['契' 'O' 'I-PER']
['尔' 'O' 'O'] ['尔' 'O' 'I-PER']
['两' 'O' 'O'] ['两' 'O' 'B-ORG']
['大' 'O' 'O'] ['大' 'O' 'I-ORG']
['党' 'O' 'O'] ['党' 'O' 'I-ORG']
['中' 'O' 'B-LOC'] ['中' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-PER']
['村' 'O' 'O'] ['村' 'O' 'I-PER']
['埔' 'O' 'I-LOC'

['长' 'O' 'O'] ['长' 'O' 'B-LOC']
['垣' 'O' 'O'] ['垣' 'O' 'I-LOC']
['县' 'O' 'O'] ['县' 'O' 'I-LOC']
['豫' 'O' 'O'] ['豫' 'O' 'B-LOC']
['北' 'O' 'O'] ['北' 'O' 'I-LOC']
['环' 'O' 'O'] ['环' 'O' 'B-ORG']
['保' 'O' 'O'] ['保' 'O' 'I-ORG']
['协' 'O' 'O'] ['协' 'O' 'I-ORG']
['陈' 'O' 'B-PER'] ['陈' 'O' 'O']
['总' 'O' 'I-PER'] ['总' 'O' 'O']
['统' 'O' 'I-PER'] ['统' 'O' 'O']
['千' 'O' 'O'] ['千' 'O' 'B-PER']
['野' 'O' 'O'] ['野' 'O' 'I-PER']
['奥' 'O' 'B-ORG'] ['奥' 'O' 'B-LOC']
['运' 'O' 'I-ORG'] ['运' 'O' 'O']
['法' 'O' 'O'] ['法' 'O' 'B-ORG']
['拉' 'O' 'O'] ['拉' 'O' 'I-ORG']
['利' 'O' 'O'] ['利' 'O' 'I-ORG']
['法' 'O' 'B-LOC'] ['法' 'O' 'B-PER']
['拉' 'O' 'I-LOC'] ['拉' 'O' 'I-PER']
['利' 'O' 'I-LOC'] ['利' 'O' 'I-PER']
['法' 'O' 'O'] ['法' 'O' 'B-ORG']
['拉' 'O' 'O'] ['拉' 'O' 'I-ORG']
['利' 'O' 'O'] ['利' 'O' 'I-ORG']
['狐' 'O' 'O'] ['狐' 'O' 'B-LOC']
['仙' 'O' 'O'] ['仙' 'O' 'I-LOC']
['陆' 'O' 'O'] ['陆' 'O' 'B-LOC']
['家' 'O' 'O'] ['家' 'O' 'I-LOC']
['嘴' 'O' 'O'] ['嘴' 'O' 'I-LOC']
['陆' 'O' 'O'] ['陆' 'O' 'B-LOC']
['家' 'O' 'O'] ['家' 'O' '

['一' 'O' 'O'] ['一' 'O' 'I-PER']
['士' 'O' 'O'] ['士' 'O' 'B-LOC']
['林' 'O' 'O'] ['林' 'O' 'I-LOC']
['英' 'O' 'O'] ['英' 'O' 'B-ORG']
['业' 'O' 'O'] ['业' 'O' 'I-ORG']
['达' 'O' 'O'] ['达' 'O' 'I-ORG']
['华' 'O' 'O'] ['华' 'O' 'B-ORG']
['国' 'O' 'O'] ['国' 'O' 'I-ORG']
['宴' 'O' 'O'] ['宴' 'O' 'I-ORG']
['陈' 'O' 'O'] ['陈' 'O' 'B-PER']
['幸' 'O' 'O'] ['幸' 'O' 'I-PER']
['妤' 'O' 'O'] ['妤' 'O' 'I-PER']
['叶' 'O' 'O'] ['叶' 'O' 'B-PER']
['国' 'O' 'O'] ['国' 'O' 'I-PER']
['华' 'O' 'O'] ['华' 'O' 'B-LOC']
['国' 'O' 'O'] ['国' 'O' 'I-LOC']
['赖' 'O' 'B-PER'] ['赖' 'O' 'O']
['建' 'O' 'I-PER'] ['建' 'O' 'B-LOC']
['洲' 'O' 'I-PER'] ['洲' 'O' 'I-LOC']
['兰' 'O' 'O'] ['兰' 'O' 'B-LOC']
['花' 'O' 'O'] ['花' 'O' 'I-LOC']
['园' 'O' 'O'] ['园' 'O' 'I-LOC']
['陈' 'O' 'O'] ['陈' 'O' 'B-PER']
['幸' 'O' 'O'] ['幸' 'O' 'I-PER']
['妤' 'O' 'O'] ['妤' 'O' 'I-PER']
['台' 'O' 'B-LOC'] ['台' 'O' 'O']
['台' 'O' 'O'] ['台' 'O' 'B-ORG']
['银' 'O' 'O'] ['银' 'O' 'I-ORG']
['台' 'O' 'B-LOC'] ['台' 'O' 'B-ORG']
['银' 'O' 'O'] ['银' 'O' 'I-ORG']
['光' 'O' 'B-LOC'] ['光' 'O' '

['河' 'O' 'O'] ['河' 'O' 'B-LOC']
['边' 'O' 'O'] ['边' 'O' 'I-LOC']
['摩' 'O' 'B-PER'] ['摩' 'O' 'B-ORG']
['根' 'O' 'I-PER'] ['根' 'O' 'I-ORG']
['士' 'O' 'I-PER'] ['士' 'O' 'I-ORG']
['丹' 'O' 'I-PER'] ['丹' 'O' 'I-ORG']
['利' 'O' 'I-PER'] ['利' 'O' 'I-ORG']
['摩' 'O' 'B-PER'] ['摩' 'O' 'B-ORG']
['根' 'O' 'I-PER'] ['根' 'O' 'I-ORG']
['士' 'O' 'I-PER'] ['士' 'O' 'I-ORG']
['丹' 'O' 'I-PER'] ['丹' 'O' 'I-ORG']
['利' 'O' 'I-PER'] ['利' 'O' 'I-ORG']
['商' 'O' 'O'] ['商' 'O' 'B-ORG']
['业' 'O' 'O'] ['业' 'O' 'I-ORG']
['行' 'O' 'O'] ['行' 'O' 'I-ORG']
['殷' 'O' 'O'] ['殷' 'O' 'B-PER']
['正' 'O' 'O'] ['正' 'O' 'I-PER']
['洋' 'O' 'O'] ['洋' 'O' 'I-PER']
['殷' 'O' 'O'] ['殷' 'O' 'B-PER']
['正' 'O' 'O'] ['正' 'O' 'I-PER']
['洋' 'O' 'O'] ['洋' 'O' 'I-PER']
['殷' 'O' 'O'] ['殷' 'O' 'B-PER']
['正' 'O' 'O'] ['正' 'O' 'I-PER']
['洋' 'O' 'O'] ['洋' 'O' 'I-PER']
['殷' 'O' 'O'] ['殷' 'O' 'B-PER']
['正' 'O' 'O'] ['正' 'O' 'I-PER']
['洋' 'O' 'O'] ['洋' 'O' 'I-PER']
['中' 'O' 'B-LOC'] ['中' 'O' 'O']
['国' 'O' 'I-LOC'] ['国' 'O' 'O']
['殷' 'O' 'O'] ['殷' 'O' 'B-PER']


['关' 'O' 'B-PER'] ['关' 'O' 'O']
['中' 'O' 'I-PER'] ['中' 'O' 'O']
['清' 'O' 'B-LOC'] ['清' 'O' 'O']
['廉' 'O' 'I-LOC'] ['廉' 'O' 'O']
['创' 'O' 'O'] ['创' 'O' 'B-ORG']
['党' 'O' 'O'] ['党' 'O' 'I-ORG']
['陈' 'O' 'O'] ['陈' 'O' 'B-PER']
['由' 'O' 'O'] ['由' 'O' 'I-PER']
['豪' 'O' 'O'] ['豪' 'O' 'I-PER']
['陈' 'O' 'O'] ['陈' 'O' 'B-PER']
['由' 'O' 'O'] ['由' 'O' 'I-PER']
['豪' 'O' 'O'] ['豪' 'O' 'I-PER']
['立' 'O' 'O'] ['立' 'O' 'B-ORG']
['院' 'O' 'O'] ['院' 'O' 'I-ORG']
['无' 'O' 'O'] ['无' 'O' 'B-ORG']
['盟' 'O' 'O'] ['盟' 'O' 'I-ORG']
['颜' 'O' 'O'] ['颜' 'O' 'B-PER']
['清' 'O' 'O'] ['清' 'O' 'I-PER']
['标' 'O' 'O'] ['标' 'O' 'I-PER']
['颜' 'O' 'O'] ['颜' 'O' 'B-PER']
['清' 'O' 'O'] ['清' 'O' 'I-PER']
['标' 'O' 'O'] ['标' 'O' 'I-PER']
['阿' 'O' 'O'] ['阿' 'O' 'B-PER']
['扁' 'O' 'O'] ['扁' 'O' 'I-PER']
['颜' 'O' 'O'] ['颜' 'O' 'B-PER']
['清' 'O' 'O'] ['清' 'O' 'I-PER']
['标' 'O' 'O'] ['标' 'O' 'I-PER']
['颜' 'O' 'O'] ['颜' 'O' 'B-PER']
['清' 'O' 'O'] ['清' 'O' 'I-PER']
['标' 'O' 'O'] ['标' 'O' 'I-PER']
['蔡' 'O' 'O'] ['蔡' 'O' 'B-PER']
['豪' 'O'

In [64]:
dif = 0
for idx in range(len(tot)):
    try:
        if tot.iloc[idx,2] != pat.iloc[idx,2]:
            print (tot.iloc[idx,:].values, pat.iloc[idx,:].values)
            dif += 1
    except:
        print (tot.iloc[idx,:].values, pat.iloc[idx,:].values)
print (dif) 

['丽' 'O' 'B-ORG'] ['丽' 'O' 'B-LOC']
['江' 'O' 'I-ORG'] ['江' 'O' 'I-LOC']
['出' 'O' 'O'] ['出' 'O' 'B-ORG']
['版' 'O' 'O'] ['版' 'O' 'I-ORG']
['局' 'O' 'O'] ['局' 'O' 'I-ORG']
['保' 'O' 'O'] ['保' 'O' 'B-ORG']
['障' 'O' 'O'] ['障' 'O' 'I-ORG']
['部' 'O' 'O'] ['部' 'O' 'I-ORG']
['保' 'O' 'O'] ['保' 'O' 'B-ORG']
['障' 'O' 'O'] ['障' 'O' 'I-ORG']
['部' 'O' 'O'] ['部' 'O' 'I-ORG']
['高' 'O' 'O'] ['高' 'O' 'B-LOC']
['中' 'O' 'O'] ['中' 'O' 'I-LOC']
['保' 'O' 'O'] ['保' 'O' 'B-ORG']
['障' 'O' 'O'] ['障' 'O' 'I-ORG']
['部' 'O' 'O'] ['部' 'O' 'I-ORG']
['小' 'O' 'O'] ['小' 'O' 'B-LOC']
['城' 'O' 'O'] ['城' 'O' 'I-LOC']
['镇' 'O' 'O'] ['镇' 'O' 'I-LOC']
['伍' 'O' 'O'] ['伍' 'O' 'B-PER']
['建' 'O' 'O'] ['建' 'O' 'I-PER']
['设' 'O' 'O'] ['设' 'O' 'I-PER']
['劳' 'O' 'B-ORG'] ['劳' 'O' 'O']
['动' 'O' 'I-ORG'] ['动' 'O' 'O']
['报' 'O' 'I-ORG'] ['报' 'O' 'O']
['小' 'O' 'B-LOC'] ['小' 'O' 'O']
['城' 'O' 'I-LOC'] ['城' 'O' 'O']
['镇' 'O' 'I-LOC'] ['镇' 'O' 'O']
['劳' 'O' 'B-ORG'] ['劳' 'O' 'O']
['动' 'O' 'I-ORG'] ['动' 'O' 'O']
['保' 'O' 'I-ORG'] ['保' 'O' 'B-OR

['人' 'O' 'O'] ['人' 'O' 'I-LOC']
['区' 'O' 'O'] ['区' 'O' 'I-LOC']
['共' 'O' 'O'] ['共' 'O' 'B-ORG']
['产' 'O' 'O'] ['产' 'O' 'I-ORG']
['共' 'O' 'O'] ['共' 'O' 'B-ORG']
['产' 'O' 'O'] ['产' 'O' 'I-ORG']
['共' 'O' 'O'] ['共' 'O' 'B-ORG']
['产' 'O' 'O'] ['产' 'O' 'I-ORG']
['党' 'O' 'O'] ['党' 'O' 'I-ORG']
['共' 'O' 'O'] ['共' 'O' 'B-ORG']
['产' 'O' 'O'] ['产' 'O' 'I-ORG']
['党' 'O' 'O'] ['党' 'O' 'I-ORG']
['田' 'O' 'B-PER'] ['田' 'O' 'O']
['里' 'O' 'I-PER'] ['里' 'O' 'O']
['劳' 'O' 'I-PER'] ['劳' 'O' 'O']
['农' 'O' 'B-PER'] ['农' 'O' 'O']
['生' 'O' 'I-PER'] ['生' 'O' 'O']
['文' 'O' 'I-PER'] ['文' 'O' 'O']
['农' 'O' 'B-PER'] ['农' 'O' 'O']
['生' 'O' 'I-PER'] ['生' 'O' 'O']
['文' 'O' 'I-PER'] ['文' 'O' 'O']
['广' 'O' 'B-LOC'] ['广' 'O' 'O']
['大' 'O' 'I-LOC'] ['大' 'O' 'O']
['龙' 'O' 'B-LOC'] ['龙' 'O' 'O']
['头' 'O' 'I-LOC'] ['头' 'O' 'O']
['农' 'O' 'B-PER'] ['农' 'O' 'O']
['生' 'O' 'I-PER'] ['生' 'O' 'O']
['文' 'O' 'I-PER'] ['文' 'O' 'O']
['台' 'O' 'O'] ['台' 'O' 'B-LOC']
['新' 'O' 'B-ORG'] ['新' 'O' 'O']
['闻' 'O' 'I-ORG'] ['闻' 'O' 'O']
['办' 'O'

['胜' 'O' 'O'] ['胜' 'O' 'I-PER']
['利' 'O' 'O'] ['利' 'O' 'I-PER']
['拉' 'O' 'O'] ['拉' 'O' 'I-PER']
['增' 'O' 'O'] ['增' 'O' 'B-LOC']
['城' 'O' 'O'] ['城' 'O' 'I-LOC']
['路' 'O' 'O'] ['路' 'O' 'I-LOC']
['边' 'O' 'O'] ['边' 'O' 'I-LOC']
['村' 'O' 'O'] ['村' 'O' 'I-LOC']
['富' 'O' 'B-LOC'] ['富' 'O' 'O']
['裕' 'O' 'I-LOC'] ['裕' 'O' 'O']
['三' 'O' 'I-LOC'] ['三' 'O' 'B-LOC']
['落' 'O' 'B-ORG'] ['落' 'O' 'O']
['实' 'O' 'I-ORG'] ['实' 'O' 'O']
['党' 'O' 'I-ORG'] ['党' 'O' 'O']
['卫' 'O' 'B-ORG'] ['卫' 'O' 'O']
['生' 'O' 'I-ORG'] ['生' 'O' 'O']
['院' 'O' 'I-ORG'] ['院' 'O' 'O']
['卫' 'O' 'B-ORG'] ['卫' 'O' 'O']
['生' 'O' 'I-ORG'] ['生' 'O' 'O']
['院' 'O' 'I-ORG'] ['院' 'O' 'O']
['国' 'O' 'O'] ['国' 'O' 'B-ORG']
['务' 'O' 'O'] ['务' 'O' 'I-ORG']
['广' 'O' 'B-ORG'] ['广' 'O' 'O']
['大' 'O' 'I-ORG'] ['大' 'O' 'O']
['辛' 'O' 'B-PER'] ['辛' 'O' 'O']
['劳' 'O' 'I-PER'] ['劳' 'O' 'O']
['扎' 'O' 'O'] ['扎' 'O' 'B-ORG']
['实' 'O' 'O'] ['实' 'O' 'I-ORG']
['朝' 'O' 'B-LOC'] ['朝' 'O' 'I-LOC']
['中' 'O' 'B-LOC'] ['中' 'O' 'B-ORG']
['阿' 'O' 'I-LOC'] ['阿' 'O' '

['、' 'O' 'B-PER'] ['、' 'O' 'O']
['永' 'O' 'I-PER'] ['永' 'O' 'B-LOC']
['和' 'O' 'I-PER'] ['和' 'O' 'I-LOC']
['华' 'O' 'B-LOC'] ['华' 'O' 'O']
['社' 'O' 'I-LOC'] ['社' 'O' 'O']
['区' 'O' 'I-LOC'] ['区' 'O' 'O']
['超' 'O' 'B-ORG'] ['超' 'O' 'O']
['大' 'O' 'I-ORG'] ['大' 'O' 'O']
['高' 'O' 'O'] ['高' 'O' 'B-ORG']
['校' 'O' 'O'] ['校' 'O' 'I-ORG']
['玛' 'O' 'B-PER'] ['玛' 'O' 'O']
['尔' 'O' 'I-PER'] ['尔' 'O' 'O']
['斯' 'O' 'I-PER'] ['斯' 'O' 'O']
['士' 'O' 'I-LOC'] ['士' 'O' 'O']
['乌' 'O' 'B-LOC'] ['乌' 'O' 'O']
['克' 'O' 'I-LOC'] ['克' 'O' 'O']
['兰' 'O' 'I-LOC'] ['兰' 'O' 'O']
['舍' 'O' 'B-PER'] ['舍' 'O' 'O']
['甫' 'O' 'I-PER'] ['甫' 'O' 'O']
['琴' 'O' 'I-PER'] ['琴' 'O' 'O']
['科' 'O' 'I-PER'] ['科' 'O' 'O']
['美' 'O' 'O'] ['美' 'O' 'B-LOC']
['意' 'O' 'B-LOC'] ['意' 'O' 'O']
['大' 'O' 'I-LOC'] ['大' 'O' 'O']
['马' 'O' 'B-LOC'] ['马' 'O' 'O']
['特' 'O' 'I-LOC'] ['特' 'O' 'O']
['拉' 'O' 'I-LOC'] ['拉' 'O' 'O']
['齐' 'O' 'I-LOC'] ['齐' 'O' 'O']
['意' 'O' 'B-LOC'] ['意' 'O' 'O']
['大' 'O' 'I-LOC'] ['大' 'O' 'O']
['凯' 'O' 'B-ORG'] ['凯' 'O' 'O']


['坛' 'O' 'I-ORG'] ['坛' 'O' 'O']
['贺' 'O' 'B-PER'] ['贺' 'O' 'O']
['礼' 'O' 'I-PER'] ['礼' 'O' 'O']
['司' 'O' 'O'] ['司' 'O' 'B-PER']
['仪' 'O' 'O'] ['仪' 'O' 'I-PER']
['本' 'O' 'O'] ['本' 'O' 'I-PER']
['嘉' 'O' 'B-LOC'] ['嘉' 'O' 'O']
['乐' 'O' 'I-LOC'] ['乐' 'O' 'O']
['仔' 'O' 'I-LOC'] ['仔' 'O' 'O']
['方' 'O' 'B-PER'] ['方' 'O' 'O']
['美' 'O' 'I-PER'] ['美' 'O' 'O']
['麦' 'O' 'B-LOC'] ['麦' 'O' 'O']
['指' 'O' 'I-LOC'] ['指' 'O' 'O']
['加' 'O' 'I-LOC'] ['加' 'O' 'O']
['东' 'O' 'O'] ['东' 'O' 'B-LOC']
['江' 'O' 'O'] ['江' 'O' 'I-LOC']
['水' 'O' 'O'] ['水' 'O' 'I-LOC']
['西' 'O' 'B-LOC'] ['西' 'O' 'O']
['隧' 'O' 'I-LOC'] ['隧' 'O' 'O']
['上' 'O' 'B-LOC'] ['上' 'O' 'O']
['石' 'O' 'I-LOC'] ['石' 'O' 'O']
['小' 'O' 'O'] ['小' 'O' 'B-LOC']
['巴' 'O' 'O'] ['巴' 'O' 'I-LOC']
['齐' 'O' 'B-LOC'] ['齐' 'O' 'B-PER']
['贝' 'O' 'I-LOC'] ['贝' 'O' 'I-PER']
['吉' 'O' 'I-LOC'] ['吉' 'O' 'I-PER']
['油' 'O' 'B-LOC'] ['油' 'O' 'O']
['源' 'O' 'I-LOC'] ['源' 'O' 'O']
['中' 'O' 'B-ORG'] ['中' 'O' 'O']
['石' 'O' 'I-ORG'] ['石' 'O' 'O']
['化' 'O' 'I-ORG'] ['化' 'O' '

['丹' 'O' 'I-ORG'] ['丹' 'O' 'I-PER']
['利' 'O' 'I-ORG'] ['利' 'O' 'I-PER']
['中' 'O' 'B-ORG'] ['中' 'O' 'O']
['行' 'O' 'I-ORG'] ['行' 'O' 'O']
['意' 'O' 'O'] ['意' 'O' 'B-LOC']
['甲' 'O' 'O'] ['甲' 'O' 'I-LOC']
['云' 'O' 'B-PER'] ['云' 'O' 'O']
['达' 'O' 'I-PER'] ['达' 'O' 'O']
['斯' 'O' 'I-PER'] ['斯' 'O' 'O']
['祖' 'O' 'B-PER'] ['祖' 'O' 'O']
['云' 'O' 'I-PER'] ['云' 'O' 'O']
['达' 'O' 'I-PER'] ['达' 'O' 'O']
['斯' 'O' 'I-PER'] ['斯' 'O' 'O']
['云' 'O' 'B-PER'] ['云' 'O' 'O']
['达' 'O' 'I-PER'] ['达' 'O' 'O']
['斯' 'O' 'I-PER'] ['斯' 'O' 'O']
['简' 'O' 'B-LOC'] ['简' 'O' 'O']
['拿' 'O' 'I-LOC'] ['拿' 'O' 'O']
['华' 'O' 'I-LOC'] ['华' 'O' 'O']
['路' 'O' 'I-LOC'] ['路' 'O' 'O']
['马' 'O' 'B-PER'] ['马' 'O' 'B-LOC']
['甸' 'O' 'I-PER'] ['甸' 'O' 'I-LOC']
['尼' 'O' 'I-PER'] ['尼' 'O' 'I-LOC']
['祖' 'O' 'B-PER'] ['祖' 'O' 'O']
['云' 'O' 'I-PER'] ['云' 'O' 'O']
['达' 'O' 'I-PER'] ['达' 'O' 'B-PER']
['拉' 'O' 'O'] ['拉' 'O' 'B-LOC']
['素' 'O' 'O'] ['素' 'O' 'I-LOC']
['比' 'O' 'B-PER'] ['比' 'O' 'B-LOC']
['哈' 'O' 'I-PER'] ['哈' 'O' 'I-LOC']
['美' 'O'

['塑' 'O' 'I-ORG'] ['塑' 'O' 'O']
['议' 'O' 'B-ORG'] ['议' 'O' 'O']
['会' 'O' 'I-ORG'] ['会' 'O' 'O']
['台' 'O' 'B-ORG'] ['台' 'O' 'O']
['塑' 'O' 'I-ORG'] ['塑' 'O' 'O']
['中' 'O' 'B-ORG'] ['中' 'O' 'O']
['油' 'O' 'I-ORG'] ['油' 'O' 'O']
['台' 'O' 'B-ORG'] ['台' 'O' 'O']
['塑' 'O' 'I-ORG'] ['塑' 'O' 'O']
['百' 'O' 'B-PER'] ['百' 'O' 'O']
['里' 'O' 'I-PER'] ['里' 'O' 'O']
['侯' 'O' 'I-PER'] ['侯' 'O' 'O']
['苗' 'O' 'O'] ['苗' 'O' 'B-PER']
['栗' 'O' 'O'] ['栗' 'O' 'I-PER']
['立' 'O' 'B-ORG'] ['立' 'O' 'O']
['委' 'O' 'I-ORG'] ['委' 'O' 'O']
['杏' 'O' 'B-PER'] ['杏' 'O' 'B-LOC']
['陵' 'O' 'I-PER'] ['陵' 'O' 'I-LOC']
['各' 'O' 'B-LOC'] ['各' 'O' 'O']
['县' 'O' 'I-LOC'] ['县' 'O' 'O']
['市' 'O' 'I-LOC'] ['市' 'O' 'O']
['青' 'O' 'O'] ['青' 'O' 'B-LOC']
['春' 'O' 'O'] ['春' 'O' 'I-LOC']
['达' 'O' 'O'] ['达' 'O' 'I-LOC']
['文' 'O' 'O'] ['文' 'O' 'B-ORG']
['课' 'O' 'O'] ['课' 'O' 'I-ORG']
['本' 'O' 'O'] ['本' 'O' 'I-ORG']
['青' 'O' 'O'] ['青' 'O' 'B-LOC']
['春' 'O' 'O'] ['春' 'O' 'I-LOC']
['达' 'O' 'O'] ['达' 'O' 'I-LOC']
['青' 'O' 'O'] ['青' 'O' 'B-LOC']


['安' 'O' 'O'] ['安' 'O' 'I-ORG']
['会' 'O' 'O'] ['会' 'O' 'I-ORG']
['中' 'O' 'I-LOC'] ['中' 'O' 'B-PER']
['村' 'O' 'I-LOC'] ['村' 'O' 'I-PER']
['中' 'O' 'B-PER'] ['中' 'O' 'B-LOC']
['村' 'O' 'I-PER'] ['村' 'O' 'I-LOC']
['中' 'O' 'O'] ['中' 'O' 'B-PER']
['村' 'O' 'O'] ['村' 'O' 'I-PER']
['台' 'O' 'B-ORG'] ['台' 'O' 'O']
['泥' 'O' 'I-ORG'] ['泥' 'O' 'O']
['联' 'O' 'B-ORG'] ['联' 'O' 'O']
['电' 'O' 'I-ORG'] ['电' 'O' 'O']
['日' 'O' 'B-LOC'] ['日' 'O' 'O']
['台' 'O' 'B-ORG'] ['台' 'O' 'O']
['积' 'O' 'I-ORG'] ['积' 'O' 'O']
['联' 'O' 'B-ORG'] ['联' 'O' 'O']
['发' 'O' 'I-ORG'] ['发' 'O' 'O']
['科' 'O' 'I-ORG'] ['科' 'O' 'O']
['北' 'O' 'B-ORG'] ['北' 'O' 'O']
['商' 'O' 'I-ORG'] ['商' 'O' 'O']
['银' 'O' 'I-ORG'] ['银' 'O' 'O']
['华' 'O' 'B-LOC'] ['华' 'O' 'B-PER']
['南' 'O' 'I-LOC'] ['南' 'O' 'I-PER']
['金' 'O' 'O'] ['金' 'O' 'I-PER']
['、' 'O' 'O'] ['、' 'O' 'I-PER']
['兆' 'O' 'B-ORG'] ['兆' 'O' 'I-PER']
['丰' 'O' 'I-ORG'] ['丰' 'O' 'O']
['智' 'O' 'B-LOC'] ['智' 'O' 'O']
['原' 'O' 'I-LOC'] ['原' 'O' 'O']
['阿' 'O' 'O'] ['阿' 'O' 'B-PER']
['扁' 'O' 'O'

['频' 'O' 'O'] ['频' 'O' 'I-ORG']
['房' 'O' 'B-ORG'] ['房' 'O' 'O']
['讯' 'O' 'I-ORG'] ['讯' 'O' 'O']
['总' 'O' 'I-ORG'] ['总' 'O' 'O']
['游' 'O' 'B-PER'] ['游' 'O' 'O']
['世' 'O' 'I-PER'] ['世' 'O' 'O']
['一' 'O' 'I-PER'] ['一' 'O' 'O']
['小' 'O' 'B-ORG'] ['小' 'O' 'O']
['华' 'O' 'I-ORG'] ['华' 'O' 'O']
['驸' 'O' 'O'] ['驸' 'O' 'B-ORG']
['马' 'O' 'O'] ['马' 'O' 'I-ORG']
['帮' 'O' 'O'] ['帮' 'O' 'I-ORG']
['全' 'O' 'B-PER'] ['全' 'O' 'O']
['家' 'O' 'I-PER'] ['家' 'O' 'O']
['福' 'O' 'I-PER'] ['福' 'O' 'O']
['小' 'O' 'B-LOC'] ['小' 'O' 'O']
['华' 'O' 'I-LOC'] ['华' 'O' 'O']
['小' 'O' 'B-LOC'] ['小' 'O' 'O']
['华' 'O' 'I-LOC'] ['华' 'O' 'O']
['小' 'O' 'B-LOC'] ['小' 'O' 'O']
['华' 'O' 'I-LOC'] ['华' 'O' 'O']
['台' 'O' 'B-LOC'] ['台' 'O' 'O']
['房' 'O' 'B-ORG'] ['房' 'O' 'O']
['讯' 'O' 'I-ORG'] ['讯' 'O' 'O']
['台' 'O' 'O'] ['台' 'O' 'B-LOC']
['训' 'O' 'O'] ['训' 'O' 'I-PER']
['台' 'O' 'B-ORG'] ['台' 'O' 'B-LOC']
['大' 'O' 'I-ORG'] ['大' 'O' 'I-LOC']
['申' 'O' 'B-ORG'] ['申' 'O' 'O']
['报' 'O' 'I-ORG'] ['报' 'O' 'O']
['法' 'O' 'I-ORG'] ['法' 'O' 'O']


['米' 'O' 'B-PER'] ['米' 'O' 'B-LOC']
['尔' 'O' 'I-PER'] ['尔' 'O' 'I-LOC']
['顿' 'O' 'I-PER'] ['顿' 'O' 'I-LOC']
['齐' 'O' 'O'] ['齐' 'O' 'B-ORG']
['发' 'O' 'O'] ['发' 'O' 'I-ORG']
['陈' 'O' 'O'] ['陈' 'O' 'B-PER']
['致' 'O' 'O'] ['致' 'O' 'I-PER']
['远' 'O' 'O'] ['远' 'O' 'I-PER']
['复' 'O' 'O'] ['复' 'O' 'I-PER']
['活' 'O' 'O'] ['活' 'O' 'I-PER']
['奇' 'O' 'B-ORG'] ['奇' 'O' 'O']
['佳' 'O' 'I-ORG'] ['佳' 'O' 'O']
['清' 'O' 'B-ORG'] ['清' 'O' 'O']
['廉' 'O' 'I-ORG'] ['廉' 'O' 'O']
['亲' 'O' 'B-ORG'] ['亲' 'O' 'O']
['民' 'O' 'I-ORG'] ['民' 'O' 'B-ORG']
['中' 'O' 'B-ORG'] ['中' 'O' 'O']
['道' 'O' 'I-ORG'] ['道' 'O' 'O']
['亲' 'O' 'B-ORG'] ['亲' 'O' 'O']
['民' 'O' 'I-ORG'] ['民' 'O' 'B-ORG']
['中' 'O' 'O'] ['中' 'O' 'B-PER']
['正' 'O' 'O'] ['正' 'O' 'I-PER']
['阿' 'O' 'B-PER'] ['阿' 'O' 'O']
['扁' 'O' 'I-PER'] ['扁' 'O' 'O']
['阿' 'O' 'B-PER'] ['阿' 'O' 'O']
['扁' 'O' 'I-PER'] ['扁' 'O' 'O']
['小' 'O' 'B-LOC'] ['小' 'O' 'O']
['马' 'O' 'I-LOC'] ['马' 'O' 'O']
['哥' 'O' 'I-LOC'] ['哥' 'O' 'O']
['亲' 'O' 'B-ORG'] ['亲' 'O' 'O']
['民' 'O' 'I-ORG'] ['

In [65]:
print (float(dif) / len(tot))

0.012685000256055718


In [27]:
# compare word in train and test
import pandas as pd
col_names_train = ['word', 'gold']
col_names_test = ['word', 'gold']
train = pd.read_csv('./train_total.jian', header=None, sep='\s+', names=col_names_train, encoding='utf-8')
test = pd.read_csv('./pred.jian', header=None, sep='\s+', names=col_names_test, encoding='utf-8')
s1 = set(train['word'].values)
s2 = set(test['word'].values)
print (len(s1), len(s2), len(s1 & s2))

4229 3599 3323


In [28]:
print (list(s1)[:10])

['尹', '吾', '侮', '讲', '撬', '侬', '灰', '翡', '疡', '瑟']


In [29]:
print (list(s2)[:10])

['尹', '吾', '娓', '侮', '嵘', '讲', '灰', '寮', '豪', '供']


In [30]:
test.head(5)

,word,gold
0,这,O
1,是,O
2,一,O
3,次,O
4,意,O
